In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
import sklearn.metrics

In [2]:
from sklearn.metrics import plot_confusion_matrix

In [3]:
from sklearn.metrics import roc_curve, auc

In [4]:
from tqdm import tqdm 

### DATASET = Original + Augmented

### MIC

### ALL

In [5]:
orig_loc = "./Data_CSV/Experiment_W_WO_CMN/"
aug_loc = "./Data_CSV/Augmneted_MFCC_data_csv/"

In [6]:
#without CMN #original_data
mic_all_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/Without_CMN_mic(16k)_all.csv")
mic_bld_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)micB_L_D_DPB.csv")
mic_blt_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)micB_L_T_DPB.csv")
mic_brd_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)micB_R_D_DPB.csv")
mic_brt_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)micB_R_T_DPB.csv")

#### Augmented Data

In [8]:
mic_bld_aug_df = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/micB_L_D_DPB.csv")
mic_blt_aug_df = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/micB_L_T_DPB.csv")
mic_brd_aug_df = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/micB_R_D_DPB.csv")
mic_brt_aug_df = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/micB_R_T_DPB.csv")
mic_all_aug_df = pd.concat([mic_bld_aug_df,mic_blt_aug_df,mic_brd_aug_df,mic_brt_aug_df])

In [9]:
#fold
def fold(sound_dataframe, sets_loc, fold_type):
    big = []
    for j in os.listdir(sets_loc):      

        file = open(sets_loc+j, 'r') 
        Lines = file.readlines() 

        count = 0
        # Strips the newline character 
        # for line in Lines: 
        #     print("Line{}: {}".format(count, line.strip())) 
        try:
            set_dataframe = pd.DataFrame()
            for i, line in enumerate(Lines):
                if fold_type == 'original':
                    A = sound_dataframe[(sound_dataframe['73'] == (Lines[i].strip()+".json"))]
                else:
                    A = sound_dataframe[(sound_dataframe['73'] == ("Aug"+Lines[i].strip()+".json"))]
                set_dataframe = pd.DataFrame.append(set_dataframe,A)
        except:
            pass
            
        
        big.append(set_dataframe)
    fold_1 = shuffle(pd.concat(big[:6]))
    test_1 = shuffle(big[6])
    fold_2 = shuffle(pd.concat(big[1:7]))
    test_2 = shuffle(big[0])
    fold_3 = shuffle(pd.concat([big[0],big[2],big[3],big[4], big[5], big[6]]))
    test_3 = shuffle(big[1])
    fold_4 = shuffle(pd.concat([big[0],big[1],big[3],big[4], big[5], big[6]]))
    test_4 = shuffle(big[2])
    fold_5 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[5], big[6]]))
    test_5 = shuffle(big[3])
    fold_6 = shuffle(pd.concat([big[0],big[2],big[1],big[3], big[5], big[6]]))
    test_6 = shuffle(big[4])
    fold_7 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[3], big[6]]))
    test_7 = shuffle(big[5])
    
    return fold_1,test_1,fold_2,test_2,fold_3,test_3,fold_4,test_4,fold_5,test_5, fold_6, test_6, fold_7,test_7

In [10]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_all_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_all_aug_df, './Sets/', 'augmented')


In [11]:
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [18]:
def SVM(fold_list):
    list_train = []
    list_test = []
    AUC = []
    for i in tqdm(range(0,len(fold_list),2)):
        fold = fold_list[i]
        test = fold_list[i+1]
        X_train = fold.iloc[:,:72]
        y_train = fold.iloc[:,72]
        X_test = test.iloc[:,:72]
        y_test = test.iloc[:,72]

        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)

        X_test = scaler.transform(X_test)

        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)


        ### SVM

        svm = SVC()

        svm.fit(X_train,y_train)
        y_score = svm.decision_function(X_test)
        fpr, tpr, thr = roc_curve(y_test, y_score)
        roc_auc = auc(fpr,tpr)
        AUC.append(roc_auc)
        #plot confusion Matrix
#         plot_confusion_matrix(svm, X_test, y_test)
#         plt.savefig('CM'+str(i)+'jpg')
        
        list_test.append(accuracy_score(y_test,svm.predict(X_test)))

        list_train.append(accuracy_score(y_train,svm.predict(X_train)))
        
    return list_test, list_train, AUC

In [14]:
test, train, AUC = SVM(fold_list)

100%|██████████| 7/7 [09:21<00:00, 81.52s/it]


In [15]:
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

### Linear Kernal 

In [16]:
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

,Train,Test,AUC
Fold1,0.798198,0.478673,0.441231
Fold2,0.761669,0.535050,0.593709
Fold3,0.751936,0.703611,0.762913
Fold4,0.770392,0.623711,0.628872
Fold5,0.792644,0.465940,0.449819
Fold6,0.794673,0.269531,0.192877
Fold7,0.758924,0.678625,0.670412
Mean,0.775491,0.536449,0.534262
STD,0.017825,0.139042,0.175512


### RBF Kernal

In [21]:
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

,Train,Test,AUC
Fold1,0.970243,0.446682,0.458887
Fold2,0.974662,0.573677,0.670973
Fold3,0.970050,0.692403,0.755829
Fold4,0.970664,0.505155,0.474450
Fold5,0.970066,0.595368,0.646362
Fold6,0.975525,0.341797,0.344557
Fold7,0.969094,0.603886,0.656502
Mean,0.971472,0.536995,0.572509
STD,0.002343,0.107339,0.136433


### BRT_MIC

In [17]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_brt_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_brt_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [18]:
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:07<00:00,  1.13s/it]


,Train,Test,AUC
Fold1,0.983763,0.443478,0.469006
Fold2,0.983399,0.534031,0.636521
Fold3,0.983740,0.555556,0.688422
Fold4,0.980769,0.537313,0.636120
Fold5,0.978292,0.605263,0.693255
Fold6,0.985130,0.580645,0.592802
Fold7,0.985435,0.679739,0.748355
Mean,0.982933,0.562289,0.637783
STD,0.002356,0.067148,0.083136


### BRD_MIC

In [19]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_brd_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_brd_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [20]:
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:08<00:00,  1.23s/it]


,Train,Test,AUC
Fold1,0.987250,0.541063,0.542657
Fold2,0.978638,0.697183,0.777073
Fold3,0.975121,0.697674,0.757572
Fold4,0.979028,0.556962,0.424451
Fold5,0.979831,0.532020,0.516145
Fold6,0.980986,0.366197,0.349559
Fold7,0.980225,0.633880,0.713613
Mean,0.980154,0.574997,0.583010
STD,0.003382,0.107150,0.156264


### BLT_Mic

In [21]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_blt_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_blt_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [22]:
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:08<00:00,  1.26s/it]


,Train,Test,AUC
Fold1,0.990643,0.334884,0.451238
Fold2,0.987678,0.569307,0.659917
Fold3,0.985885,0.655660,0.713741
Fold4,0.987635,0.682759,0.785388
Fold5,0.984390,0.608696,0.687381
Fold6,0.990667,0.265152,0.202770
Fold7,0.987385,0.694611,0.776778
Mean,0.987755,0.544438,0.611030
STD,0.002131,0.160676,0.195978


### BLD_MIC

In [23]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_bld_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_bld_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [24]:
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:08<00:00,  1.16s/it]


,Train,Test,AUC
Fold1,0.983707,0.453125,0.504687
Fold2,0.981188,0.695122,0.745283
Fold3,0.981205,0.413793,0.419006
Fold4,0.979349,0.296552,0.262938
Fold5,0.978550,0.543590,0.553715
Fold6,0.985613,0.210526,0.126014
Fold7,0.981491,0.397516,0.478365
Mean,0.981586,0.430032,0.441430
STD,0.002244,0.146883,0.185992


### EKO

### ALL

In [25]:
#without CMN #original_data
eko_all_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/Without_CMN_eko_all.csv")
eko_bld_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)ekoB_L_D_DPB.csv")
eko_blt_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)ekoB_L_T_DPB.csv")
eko_brd_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)ekoB_R_D_DPB.csv")
eko_brt_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)ekoB_R_T_DPB.csv")
eko_bld_aug_df   = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/ekoB_L_D_DPB.csv")
eko_blt_aug_df   = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/ekoB_L_T_DPB.csv")
eko_brd_aug_df   = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/ekoB_R_D_DPB.csv")
eko_brt_aug_df   = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/ekoB_R_T_DPB.csv")
eko_all_aug_df   = pd.concat([eko_bld_aug_df,eko_blt_aug_df,eko_brd_aug_df,eko_brt_aug_df]) 

In [26]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_all_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_all_aug_df, './Sets/', 'augmented')


In [27]:
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [28]:
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:58<00:00,  8.41s/it]


,Train,Test,AUC
Fold1,0.902790,0.560427,0.585044
Fold2,0.902934,0.596567,0.709513
Fold3,0.901176,0.485679,0.480710
Fold4,0.903252,0.678694,0.712578
Fold5,0.894314,0.678474,0.765301
Fold6,0.916022,0.523438,0.665976
Fold7,0.905726,0.626308,0.659064
Mean,0.903745,0.592798,0.654027
STD,0.006000,0.068857,0.087713


### BRT_eko

In [29]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_brt_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_brt_aug_df, './Sets/', 'augmented')


In [30]:
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [31]:
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:03<00:00,  1.91it/s]


,Train,Test,AUC
Fold1,0.947938,0.604348,0.581131
Fold2,0.933598,0.738220,0.824671
Fold3,0.933943,0.648148,0.729674
Fold4,0.938086,0.589552,0.560089
Fold5,0.927481,0.598684,0.652159
Fold6,0.943773,0.653226,0.646253
Fold7,0.936008,0.673203,0.710526
Mean,0.937261,0.643626,0.672072
STD,0.006310,0.048435,0.084479


### BRD_eko

In [32]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_brd_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_brd_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:03<00:00,  1.98it/s]


,Train,Test,AUC
Fold1,0.943500,0.714976,0.763437
Fold2,0.944601,0.450704,0.681936
Fold3,0.940097,0.424419,0.364927
Fold4,0.932793,0.746835,0.852107
Fold5,0.930777,0.482759,0.419860
Fold6,0.945540,0.521127,0.746394
Fold7,0.946289,0.459016,0.423382
Mean,0.940514,0.542834,0.607435
STD,0.005841,0.122396,0.183976


### BLT_eko

In [33]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_blt_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_blt_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:03<00:00,  1.75it/s]


,Train,Test,AUC
Fold1,0.936660,0.553488,0.538952
Fold2,0.953555,0.465347,0.577200
Fold3,0.947368,0.292453,0.277359
Fold4,0.950090,0.648276,0.789193
Fold5,0.940820,0.565217,0.663452
Fold6,0.959111,0.393939,0.537855
Fold7,0.954587,0.700599,0.783828
Mean,0.948885,0.517046,0.595406
STD,0.007340,0.132466,0.162863


### BLD_eko 

In [34]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_bld_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_bld_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]
test, train, AUC = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

AUC.append(np.mean(AUC))
AUC.append(np.std(AUC[:7]))

pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1), np.array(AUC).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test', 'AUC'])

100%|██████████| 7/7 [00:04<00:00,  1.79it/s]


,Train,Test,AUC
Fold1,0.947047,0.427083,0.401897
Fold2,0.950990,0.591463,0.688842
Fold3,0.949537,0.334975,0.261696
Fold4,0.941691,0.620690,0.635274
Fold5,0.946374,0.651282,0.753953
Fold6,0.960377,0.421053,0.701014
Fold7,0.956071,0.503106,0.524840
Mean,0.950298,0.507093,0.566788
STD,0.005807,0.109713,0.166761
